Instalamos las dependencias

In [12]:
!pip install requests
!pip install BeautifulSoup4
!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
!pip install langchain-ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
import requests
from bs4 import BeautifulSoup
from langchain_ollama.chat_models import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Lionel_Messi'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    page_text = soup.get_text(separator=' ', strip=True)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
    web_texts = text_splitter.split_text(page_text)
    print(web_texts)
else:
    print(f'No se pudo acceder a la página: {response.status_code}')

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [18]:
bbdd_vector = Chroma.from_texts(
    texts=web_texts,
    collection_name="players",
    embedding=embeddings,
)

In [19]:
retriever = bbdd_vector.as_retriever()

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = "llama3.2"
modelo = ChatOllama(model=llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}| prompt| modelo| StrOutputParser()
)
chain.invoke("Who is Lionel Messi?")

"Lionel Messi is an Argentine international who plays as a forward, widely regarded as one of the greatest players of all time. He has won numerous records for individual accolades, including eight Ballon d'Or awards and eight times being named the world's best player by FIFA."